In [1]:
import urllib.request

urllib.request.urlretrieve('https://archive.ics.uci.edu/static/public/53/iris.zip',
                          'data.zip')

('data.zip', <http.client.HTTPMessage at 0x7f202459ad40>)

In [2]:
# Create a directory for extracted data
!mkdir data
!unzip data.zip -d data/

Archive:  data.zip
  inflating: data/Index              
  inflating: data/bezdekIris.data    
  inflating: data/iris.data          
  inflating: data/iris.names         


In [3]:
#read data
import pandas as pd
data = pd.read_csv('data/iris.data', header=None)

#convert to numerical values 
data[4] = data[4].replace('Iris-setosa',0)
data[4] = data[4].replace('Iris-virginica',1)
data[4] = data[4].replace('Iris-versicolor',2)

#shuffle

data = data.sample(frac=1).reset_index(drop=True)

#change label column index

data = data[[4,0,1,2,3]]

#split

#80% train data
#20% validation data

train_data = data[:120]
val_data = data[120:]


In [4]:
import boto3

bucket_name = 'sagemaker-build-and-deploy-sagemaker'

train_data.to_csv('data.csv',header=False, index = False)
key = 'data/train/data'
url = 's3://{}/{}'.format(bucket_name,key)
boto3.Session().resource('s3').Bucket(bucket_name).Object(key).upload_file('data.csv')

val_data.to_csv('data.csv',header=False, index = False)
key = 'data/val/data'
url = 's3://{}/{}'.format(bucket_name,key)
boto3.Session().resource('s3').Bucket(bucket_name).Object(key).upload_file('data.csv')

In [8]:
import sagemaker
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker import get_execution_role

key = 'model/xgb_model'
s3_output_location = url = 's3://{}/{}'.format(bucket_name,key)

xgb_model = sagemaker.estimator.Estimator(
    get_image_uri(boto3.Session().region_name,'xgboost'),
    get_execution_role(),
    train_instance_count = 1,
    train_instance_type = 'ml.m5.xlarge',
    train_volume_size = 5,
    output_path = s3_output_location,
    sagemaker_session = sagemaker.Session()
)

xgb_model.set_hyperparameters(max_depth = 5,
                              eta = 0.2,
                              gamma = 4,
                              min_child_weight = 6,
                              silent = 0,
                              objective = 'multi:softmax',
                              num_class = 3,
                              num_round=10)


[01/05/25 04:50:37] WARNING  The method get_image_uri has been renamed in sagemaker>=2.          ]8;id=622060;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py\deprecations.py]8;;\:]8;id=699647;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py#34\34]8;;\
                             See: https://sagemaker.readthedocs.io/en/stable/v2.html for                           
                             details.                                                                              

                    INFO     Ignoring unnecessary instance type: None.                            ]8;id=110287;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=148175;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#524\524]8;;\

                    WARNING  train_instance_count has been renamed in sagemaker>=2.              ]8;id=293391;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py\deprecations.py]8;;\:]8;id=836898;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py#34\34]8;;\
                             See: https://sagemaker.readthedocs.io/en/stable/v2.html for                           
                             details.                                                                              

                    WARNING  train_instance_type has been renamed in sagemaker>=2.               ]8;id=185379;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py\deprecations.py]8;;\:]8;id=139862;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py#34\34]8;;\
                             See: https://sagemaker.readthedocs.io/en/stable/v2.html for                           
                             details.                                                                              

                    WARNING  train_volume_size has been renamed in sagemaker>=2.                 ]8;id=602480;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py\deprecations.py]8;;\:]8;id=979963;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py#34\34]8;;\
                             See: https://sagemaker.readthedocs.io/en/stable/v2.html for                           
                             details.                                                                              

In [9]:
train_data = 's3://{}/{}'.format(bucket_name,'data/train')
val_data = 's3://{}/{}'.format(bucket_name,'data/val')

train_channel = sagemaker.session.s3_input(train_data,content_type='text/csv') 
val_channel = sagemaker.session.s3_input(val_data,content_type='text/csv') 

data_channels = {'train' : train_channel , 'validation' : val_channel}

xgb_model.fit(inputs=data_channels)

[01/05/25 04:50:41] WARNING  The class sagemaker.session.s3_input has been renamed in            ]8;id=147103;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py\deprecations.py]8;;\:]8;id=478959;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py#34\34]8;;\
                             sagemaker>=2.                                                                         
                             See: https://sagemaker.readthedocs.io/en/stable/v2.html for                           
                             details.                                                                              

                    WARNING  The class sagemaker.session.s3_input has been renamed in            ]8;id=571569;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py\deprecations.py]8;;\:]8;id=82888;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py#34\34]8;;\
                             sagemaker>=2.                                                                         
                             See: https://sagemaker.readthedocs.io/en/stable/v2.html for                           
                             details.                                                                              

                    INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=698095;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=452955;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#90\90]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name: xgboost-2025-01-05-04-50-41-874       ]8;id=842649;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=640537;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\

2025-01-05 04:50:45 Starting - Starting the training job...
2025-01-05 04:50:59 Starting - Preparing the instances for training...
2025-01-05 04:51:44 Downloading - Downloading the training image......
2025-01-05 04:52:40 Training - Training image download completed. Training in progress.
2025-01-05 04:52:40 Uploading - Uploading generated training modelArguments: train
[2025-01-05:04:52:32:INFO] Running standalone xgboost training.
[2025-01-05:04:52:32:INFO] File size need to be processed in the node: 0.0mb. Available memory size in the node: 8151.42mb
[2025-01-05:04:52:32:INFO] Determined delimiter of CSV input is ','
[04:52:32] S3DistributionType set as FullyReplicated
[04:52:32] 120x4 matrix with 480 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2025-01-05:04:52:32:INFO] Determined delimiter of CSV input is ','
[04:52:32] S3DistributionType set as FullyReplicated
[04:52:32] 30x4 matrix with 120 entries loaded from /opt/ml/input/data/validation?

In [10]:
xgb_predictor = xgb_model.deploy(initial_instance_count=1,
                                 instance_type = 'ml.m5.xlarge')

[01/05/25 04:56:00] INFO     Creating model with name: xgboost-2025-01-05-04-56-00-544              ]8;id=969616;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=530072;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\

[01/05/25 04:56:01] INFO     Creating endpoint-config with name xgboost-2025-01-05-04-56-00-544     ]8;id=948032;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=225118;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#5889\5889]8;;\

                    INFO     Creating endpoint with name xgboost-2025-01-05-04-56-00-544            ]8;id=363173;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=549928;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4711\4711]8;;\

------!